# Hotspot Prediction with Google Earth Engine (GEE) and LSTM

**Scope:** End-to-end template in Python for:
1) Authenticating & reading data from GEE
2) Building weekly features for a province-level AOI (Nan, Thailand)
3) Creating a supervised dataset
4) Training an LSTM to forecast hotspots
5) Evaluating and forecasting for 2025

> **Tip:** Run this notebook in Google Colab for smooth GEE auth and Drive access. On first run, you'll be prompted to authorize Earth Engine.

In [ ]:
# !pip install -q earthengine-api geemap pandas numpy matplotlib scikit-learn tensorflow
import ee, json, datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
print(tf.__version__)

## 1) Authenticate & Initialize Earth Engine

In [ ]:
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()
print('GEE initialized')

## 2) Define area of interest (AOI): Nan Province, Thailand

We try FAO GAUL level-1 for provinces. If GAUL is unavailable in your project, switch to GADM or use your own shapefile.

In [ ]:
def get_nan_province():
    # Option A: FAO GAUL simplified level 1
    try:
        gaul1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
        nan_fc = gaul1.filter(ee.Filter.And(
            ee.Filter.eq('ADM0_NAME', 'Thailand'),
            ee.Filter.eq('ADM1_NAME', 'Nan')
        ))
        geom = ee.Feature(nan_fc.first()).geometry()
        return geom
    except Exception:
        pass
    # Option B: GAUL simplified 500m level 1
    gaul1s = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level1')
    nan_fc = gaul1s.filter(ee.Filter.And(
        ee.Filter.eq('ADM0_NAME', 'Thailand'),
        ee.Filter.eq('ADM1_NAME', 'Nan')
    ))
    return ee.Feature(nan_fc.first()).geometry()

AOI = get_nan_province()
AOI

## 3) Configure time windows

We aggregate weekly features for 2020–2024 and predict 2025.

In [ ]:
train_start = ee.Date('2020-01-01')
train_end   = ee.Date('2024-12-31')
test_start  = ee.Date('2025-01-01')
test_end    = ee.Date('2025-12-31')

WEEK_MILLIS = 7 * 24 * 60 * 60 * 1000

## 4) Data sources and feature engineering

Targets (hotspots):
- **VIIRS Fire Mask** daily product (example): `NOAA/VIIRS/001/VNP14A1` (band `FireMask`). We count pixels indicating active fire per week.

Predictors:
- **NDVI/NDMI** from Landsat-8/9 SR or MODIS (MOD13Q1). Here we use MODIS 16-day NDVI/NDMI proxies and resample.
- **Topography**: SRTM slope.
- **Climate**: ERA5-Land daily (temperature, total precipitation). We weekly-aggregate.

👉 *You may switch to your preferred datasets; the logic stays the same.*

In [ ]:
# --- Helper: weekly date list ---
def weekly_dates(start, end):
    start = ee.Date(start)
    end = ee.Date(end)
    n_weeks = end.difference(start, 'week').toInt()
    def make_week(i):
        s = start.advance(i, 'week')
        e = s.advance(1, 'week')
        return ee.Feature(None, {'start': s, 'end': e})
    return ee.FeatureCollection(ee.List.sequence(0, n_weeks.subtract(1)).map(make_week))

# --- Target: VIIRS VNP14A1 FireMask (daily) -> weekly fire pixel count ---
viirs = ee.ImageCollection('NOAA/VIIRS/001/VNP14A1')

def weekly_fire_count(start, end, geom):
    # FireMask > 7 typically indicates detected fire; adjust as needed
    col = viirs.filterDate(start, end).select('FireMask')
    # Convert to binary fire image per day
    def to_fire(img):
        fire = img.gte(7).selfMask()
        return fire.rename('fire')
    daily = col.map(to_fire)
    # Sum over the week (count of fire pixels appearances)
    summed = daily.sum()
    # Pixel area to scale counts if desired (optional)
    count = summed.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geom, scale=500, maxPixels=1e13
    )
    return ee.Number(count.get('fire')).unbound()

# --- Predictors ---
# NDVI from MODIS (MOD13Q1.061) 16-day, 250m
modis_ndvi = ee.ImageCollection('MODIS/061/MOD13Q1').select('NDVI')

def weekly_ndvi_mean(start, end, geom):
    col = modis_ndvi.filterDate(start, end)
    mean = col.mean().multiply(0.0001)  # scale factor
    val = mean.reduceRegion(ee.Reducer.mean(), geom, scale=500, maxPixels=1e13)
    return ee.Number(val.get('NDVI'))

# NDMI proxy from MODIS NIR/SWIR (using MOD09A1 SR 500m 8-day)
modis_sr = ee.ImageCollection('MODIS/061/MOD09A1').select(['sur_refl_b02','sur_refl_b06'])
def weekly_ndmi_mean(start, end, geom):
    col = modis_sr.filterDate(start, end)
    def ndmi(img):
        nir = img.select('sur_refl_b02').multiply(0.0001)
        swir = img.select('sur_refl_b06').multiply(0.0001)
        n = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI')
        return n
    ndmi_col = col.map(ndmi)
    mean = ndmi_col.mean()
    val = mean.reduceRegion(ee.Reducer.mean(), geom, scale=500, maxPixels=1e13)
    return ee.Number(val.get('NDMI'))

# Slope from SRTM
srtm = ee.Image('USGS/SRTMGL1_003')
slope = ee.Terrain.slope(srtm)
def slope_mean(geom):
    val = slope.reduceRegion(ee.Reducer.mean(), geom, scale=500, maxPixels=1e13)
    return ee.Number(val.get('slope'))

# ERA5-Land: daily temperature (K) and total precipitation (m)
era5 = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY')
def weekly_era5(start, end, geom):
    col = era5.filterDate(start, end)
    t2m = col.select('temperature_2m').mean()
    tp = col.select('total_precipitation').sum()
    t_mean = t2m.reduceRegion(ee.Reducer.mean(), geom, scale=1000, maxPixels=1e13).get('temperature_2m')
    p_sum  = tp.reduceRegion(ee.Reducer.mean(), geom, scale=1000, maxPixels=1e13).get('total_precipitation')
    return ee.Dictionary({'t2m_mean': t_mean, 'tp_sum': p_sum})

# Precompute static slope
SLOPE_MEAN = slope_mean(AOI)

### Build weekly feature table
For each week we compute:
- `y_fire`: count of active-fire pixels (target)
- `ndvi_mean`, `ndmi_mean`
- `t2m_mean` (K), `tp_sum` (m)
- `slope_mean` (static)

In [ ]:
def build_weekly_df(start, end, geom):
    weeks = weekly_dates(start, end)
    def per_week(f):
        s = ee.Date(f.get('start'))
        e = ee.Date(f.get('end'))
        y = weekly_fire_count(s, e, geom)
        ndvi = weekly_ndvi_mean(s, e, geom)
        ndmi = weekly_ndmi_mean(s, e, geom)
        clim = weekly_era5(s, e, geom)
        return ee.Feature(None, {
            'start': s.format('YYYY-MM-dd'),
            'end': e.format('YYYY-MM-dd'),
            'y_fire': y,
            'ndvi_mean': ndvi,
            'ndmi_mean': ndmi,
            't2m_mean': clim.get('t2m_mean'),
            'tp_sum': clim.get('tp_sum'),
            'slope_mean': SLOPE_MEAN
        })
    fc = weeks.map(per_week)
    # Download to client as a list of dicts
    rows = fc.getInfo()['features']
    recs = [r['properties'] for r in rows]
    df = pd.DataFrame(recs)
    df['start'] = pd.to_datetime(df['start'])
    df['end'] = pd.to_datetime(df['end'])
    df = df.sort_values('start').reset_index(drop=True)
    return df

df_train = build_weekly_df(train_start, train_end, AOI)
df_test  = build_weekly_df(test_start, test_end, AOI)
df_train.head(), df_train.shape, df_test.shape

## 5) Prepare sequences for LSTM

- We use a sliding window of `seq_len` weeks to predict next-week hotspots.
- Feature scaling is applied to predictors and the target.

You can add more lags/exogenous features as needed.

In [ ]:
features = ['ndvi_mean','ndmi_mean','t2m_mean','tp_sum','slope_mean','y_fire']
seq_len = 8  # 8-week lookback

def make_sequences(df, seq_len, feature_cols, target_col='y_fire'):
    values = df[feature_cols].astype(float).values
    scaler = MinMaxScaler()
    values_scaled = scaler.fit_transform(values)
    Xs, ys = [], []
    for i in range(len(values_scaled) - seq_len):
        Xs.append(values_scaled[i:i+seq_len, :])
        ys.append(values_scaled[i+seq_len, feature_cols.index(target_col)])
    X = np.array(Xs)
    y = np.array(ys)
    return X, y, scaler

X_train, y_train, scaler = make_sequences(df_train, seq_len, features)
print(X_train.shape, y_train.shape)

## 6) Define & train the LSTM model

In [ ]:
tf.keras.backend.clear_session()
n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(n_timesteps, n_features)),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse')
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=16, verbose=1)

plt.figure()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend(); plt.title('Loss'); plt.xlabel('epoch'); plt.ylabel('MSE'); plt.show()

## 7) Evaluate on 2024 holdout (last few weeks) and Forecast 2025

We build rolling sequences across the boundary from 2024 into 2025 and inverse-transform only the target dimension.

In [ ]:
def scale_with_existing(df, scaler, feature_cols):
    vals = df[feature_cols].astype(float).values
    return scaler.transform(vals)

def build_sequences_from_scaled(scaled_vals, seq_len, target_index):
    Xs, ys = [], []
    for i in range(len(scaled_vals) - seq_len):
        Xs.append(scaled_vals[i:i+seq_len, :])
        ys.append(scaled_vals[i+seq_len, target_index])
    return np.array(Xs), np.array(ys)

TARGET_IDX = features.index('y_fire')

# Evaluate on tail of train (e.g., last 20% already handled by val). Here we proceed to forecasting on 2025.
# To forecast next-week iteratively in 2025, we combine train+test features and slide a window.
df_all = pd.concat([df_train, df_test], ignore_index=True)
scaled_all = scale_with_existing(df_all, scaler, features)
X_all, y_all = build_sequences_from_scaled(scaled_all, seq_len, TARGET_IDX)

# Predictions for all available windows; extract the windows that end in 2025
pred_scaled = model.predict(X_all)

# Inverse transform: construct an array with same feature width, put preds in target column
def inverse_target_only(pred_scaled, scaler, target_index):
    dummy = np.zeros((len(pred_scaled), len(features)))
    dummy[:, target_index] = pred_scaled.flatten()
    inv = scaler.inverse_transform(dummy)
    return inv[:, target_index]

pred_y = inverse_target_only(pred_scaled, scaler, TARGET_IDX)

# Align predictions to dates: prediction at position i corresponds to week index i+seq_len
dates = df_all['end'].values
pred_dates = dates[seq_len:]

pred_df = pd.DataFrame({'date': pred_dates, 'y_hat': pred_y})

# Merge actuals (if available)
actual = df_all.iloc[seq_len:][['end','y_fire']].reset_index(drop=True).rename(columns={'end':'date'})
eval_df = pred_df.merge(actual, on='date', how='left')

# Split to 2025 subset
eval_df['date'] = pd.to_datetime(eval_df['date'])
df_2025 = eval_df[eval_df['date'].dt.year == 2025].copy()
df_2025.head()

### Metrics (if you have actual 2025 data)
If actual 2025 targets exist (once the period passes), compute metrics; otherwise, visualize forecasts.

In [ ]:
has_actuals = df_2025['y_fire'].notna().any()
if has_actuals:
    mse = mean_squared_error(df_2025['y_fire'], df_2025['y_hat'])
    mae = mean_absolute_error(df_2025['y_fire'], df_2025['y_hat'])
    r2  = r2_score(df_2025['y_fire'], df_2025['y_hat'])
    print('MSE:', mse, 'MAE:', mae, 'R2:', r2)
else:
    print('No 2025 actuals available for metrics. Showing forecast only.')

plt.figure()
plt.plot(df_2025['date'], df_2025['y_hat'], label='Forecast (y_hat)')
if has_actuals:
    plt.plot(df_2025['date'], df_2025['y_fire'], label='Actual (y)')
plt.legend(); plt.title('Weekly Hotspot Forecast – 2025 (Nan, TH)')
plt.xlabel('date'); plt.ylabel('weekly hotspot count')
plt.xticks(rotation=45); plt.tight_layout(); plt.show()

## 8) Next steps & options
- Replace MODIS with Landsat 8/9 SR (NDVI, NDMI from NIR/SWIR) for higher spatial detail.
- Add drought indices (SPEI), relative humidity, wind speed from ERA5.
- Use **ConvLSTM** or Temporal Convolutional Networks if you move to gridded predictors instead of province averages.
- Export an example dataset to CSV for audit:
```python
df_train.to_csv('nan_weekly_train.csv', index=False)
df_test.to_csv('nan_weekly_test.csv', index=False)
```